In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
_query = """
SELECT
  run,
  trim('"' FROM seq::text )::int as seq,
  trim('"' FROM block::text )::int as block,
  trim('"' FROM min::text )::int as min,
  --e0,
  e1::decimal/100000000 as batch_sec, -- time in sec as
  --e2
  ts::text::time

FROM
  (SELECT
    run,
    e->'ts' as ts,
    e->'seq' as seq,
    e->'log'->'height' as block,
    e->'log'->'min' as min,
    --e->'log'->'event' as event,
    e->'log'->'event'->0 as e0,
    e->'log'->'event'->1 as e1,
    e->'log'->'event'->2 as e2
  FROM
    fnode0.simtest) as s
WHERE
  e2::text like '%RUNTIME%'
"""

_order = """
ORDER BY
  run, seq ;
"""

def query(i):
    return _query + ' AND run = %s ' % i + _order

connection = pg.connect("host='localdb' dbname=load user=load password='load'")

In [21]:
log_runs = pd.read_sql_query('SELECT * from log_runs where ID in (3, 1);', con=connection)
log_runs

,id,label,ts
0,1,v5_test_LLLLLLLFF_d100,2019-01-31 17:07:33.116239
1,3,v6_test_LLLLLLLFF_d100,2019-01-31 17:09:05.469478


In [28]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
frames = []
for i, n in log_runs['label'].items():
    r = log_runs['id'][i]
    df = pd.read_sql_query(query(r), con=connection)
    frames.append(df)
    data.append(
        go.Scatter(x=df['ts'], y=df['batch_sec'], name=n.replace('test_','')+"."+i.__str__())
    )

py.iplot(data, filename='WIP-simtest')

In [45]:
s = (frames[0].sum()['batch_sec'] / 10, frames[1].sum()['batch_sec'] / 10)

100 * (s[0] - s[1]) / s[1] # Average % improvement

18.907483701952447

In [46]:
s = (frames[0].max()['batch_sec'] / 10, frames[1].max()['batch_sec'] / 10)

100 * (s[0] - s[1]) / s[1] # Outlier Max % improvement

41.73539917159482

## Holding Queue Backlog Recovery


### Test Scenario: 7 Leaders 2 Followers

* Ran simulator pushing 10 sets of 1000 commit/reveal messages.
* Simulator is configured to drop 10% of network messages
* Network is configured in 'alot' configuration
* Each set is pushed into the API Queue first (without EC funding)
* Then the associated EC address is funded allowing messages to be processed.

NOTE: this test is limited by the stability of V5.1.0 we are not able to gather accurate data
for a realistic size network using only the simulator.

### Result: Speed Improvement

Each data point represents the time that it takes for the Holding Queue to return to zero. (lower is better)

This graph shows an improvement to overall Rate of Commit/Reveal message processing.

### Future Work: Repeat on Testnet

One standard metric being touted by various blockchains is Transactions-Per-Second TPS

Because we are using the simulator - this data is not appropriate to compute an overall TPS metric.

Currently we are working w/ our own Development Network.

Once we have designed some reasonably 'safe' tests to run,
we will use this technique to study Testnet.
